# Utilizing RAS-Commander for HEC-RAS Automation 

In this portion of the workshop we will use LLM's to use the new RAS-Commander Library to directly test the Manning's n function that we created in the first hour of the workshop.  


Workshop users can either download the full repository:  
[RAS-Commander Github Repository](https://github.com/gpt-cmdr/ras-commander/tree/main)  

![Download RAS-Commander Repository](images/rascmdr-downloadrepo.png)  

Or download the LLM knowledge base for RAS-Commander and utilize Google AI Studio:  
[RAS-Commander API Reference Markdown File](https://github.com/gpt-cmdr/ras-commander/tree/main/ai_tools/llm_knowledge_bases) 

Due to the size of the code base, your LLM may not be able to handle the full repositorys contents.  


NOTE: To avoid delays later in the presentation due to wifi speed, download the HEC-RAS Example Projects .zip to the ASFPM-LLM-Data-Management-Workshop folder: 
https://github.com/HydrologicEngineeringCenter/hec-downloads/releases/download/1.0.33/Example_Projects_6_6.zip  
(The ras-commander library will use this file automatically as long as it is in the same folder as the notebook)


Or run the code cell below:

In [ ]:
# Optional: Let the HEC-RAS Example Projects begin to download while we continue the presentation:
!curl -L -o Example_Projects_6_6.zip https://github.com/HydrologicEngineeringCenter/hec-downloads/releases/download/1.0.33/Example_Projects_6_6.zip

## Summary of Pre-Compiled LLM Knowledge Bases for ras-commander:

**ras-commander_fullrepo.txt** _~650k Tokens_ - All contents of the repository, all functions, cleaned notebooks, and documentation

**ras_commander_documentation_only.txt** _~188k Tokens_ - Repository Documntation, without Code.  

**ras_commander_classes_and_functions_only.txt** _~137k Tokens_ - Includes just the code.  Combine this with an example notebook in Claude or ChatGPT's o1/o3 model to ask questions and request revisions or additions

**ras_commander_ras_functions_only.txt** _~80k tokens_ - Includes only the code for RAS automation functions

ras_commander_hdf_functions_only.txt 90k** _~89k tokens_ - Includes only the code for HDF data access

## How to Leverage LLM's with RAS-Commander

- [RAS-Commander GPT](https://chatgpt.com/g/g-TZRPR3oAO-ras-commander-library-assistant)
    Similar to the GPT's we have built in this class, this GPT has access to the RAS-Commander codebase and documentation, and can perform limited operations using ChatGPT's Code Interpreter.  2 Example projects are included. 

- [API Documentation](https://github.com/gpt-cmdr/ras-commander/blob/main/api.md)(shown above)
    This document provides the minimum documentation needed for an LLM to translate your plain language commands to a series of python function calls that interact with HEC-RAS Files.

    
- Not covered in this course:

    - [Detailed Knowledge Bases](https://github.com/gpt-cmdr/ras-commander/tree/main/ai_tools/llm_knowledge_bases)  
    These knowledge bases comprise of various combinations of plain text files from the repository to provide more comprehensive context.  These are suitable for more advanced development where you want the LLM to see the underlying logic to create new functions or develop new features. 

    - [Cursor IDE Integration](https://github.com/gpt-cmdr/ras-commander/blob/main/.cursor/rules/ras-commander.mdc)
        For users that have the AI-powered VS Code Fork Cursor, there is a .cursorrules file included in the repository that automatically loads system instructions when making requests


    - [RAS-Commander Library Assistant](https://github.com/gpt-cmdr/ras-commander/tree/main/ai_tools/library_assistant)
        For users that want to use RAS-Commander with a pay-as-you-go API key from OpenAI, Anthropic, or other providers.  


## Using the RAS-Commander GPT Assistant

Before starting on our notebook, use the GPT Assistant to explore our files 


![RAS-Commander GPT Assistant Exporatory Prompts](images/rascmdr_gpt_defaults.png)


The GPT has a copy of the results from 2 HEC-RAS Example Projects (Bald Eagle Creek and the Pipes Beta project).  However GPT's are not yet reliable enough for heavy lifting with the library.  

Workshop users are invited to take 2-3 minutes to upload the `RAS_Muncie.p04.hdf` file and ask for a summary of project information and request geometric data or time series results.  These requests may or not be successful, and we will continue with oher methods to improve responses and perform the Manning's N automation. 

-----

## Proposed Workflow Description

Here we will describe the workflow that will be required to implement and test the Manning's N Base Overrides and Regional Override functions, using the HEC-RAS Example Project "BaldEagleCrkMulti2D", which has the project name "BaldEagleDamBrk"

### Detailed Workflow

    1. Create "RAS-Commander Test" subfolder in the same directory as this notebook, then extract "BaldEagleMulti2D"
    2. List all project dataframes for the user's review
    3. Existing Plan 06 uses Geometry 09.  Both of these will need to be cloned and the new plan will need to have the new geomery set as the active geometry
    4. In the new plan and geometry, revise the Plan Name, ShortID and Description to denote that all roughness tables are increased by 0.01
    5. Using the functions below, revise all base and region overrides in the new plan
    6. Run both plans, parallel, 2 cores apiece
    7. Using the point (2081626, 366459), find the nearest mesh cell and compare the results for the Template and Revised Plan.  Create a plot showing water surface elevation a the closest mesh cell. 

Work step by step through the notebook, including Markdown cells that explain each RAS-Commander function, its arguments, and how we are using it in our notebook.  Clearly explain the notebook's operation and show all work to maintain transparent operation.  

Favor simple and concise code, and short segments of code that are easy to understand, to allow the user to observe each step.  



-----

## Reference Functions from Previous Notebook

The following function is from a previous notebook, it is tested and works. 



In [3]:
# Manning's N Override Functions to use in this notebook. 
import pandas as pd
from pathlib import Path





# === Parsing functions ===
def parse_lcmann_table(lines):
    table_data = []
    current_table_number = None

    for i, line in enumerate(lines):
        if line.startswith("LCMann Table="):
            current_table_number = line.split("=")[-1].strip()
            j = i + 1
            while j < len(lines):
                next_line = lines[j].strip()
                if next_line.startswith("LCMann") or not next_line or "=" in next_line:
                    break
                if ',' in next_line:
                    try:
                        land_cover, n_val = next_line.rsplit(",", 1)
                        table_data.append([current_table_number, land_cover.strip(), float(n_val.strip())])
                    except ValueError:
                        pass
                j += 1
            break

    return pd.DataFrame(table_data, columns=["Table", "Land Cover Type", "n"])

def parse_lcmann_region_table(lines):
    table_data = []
    current_table_number = None

    for i, line in enumerate(lines):
        if line.startswith("LCMann Region Table="):
            current_table_number = line.split("=")[-1].strip()
            j = i + 1
            while j < len(lines):
                next_line = lines[j].strip()
                if next_line.startswith("LCMann") or not next_line or "=" in next_line:
                    break
                if ',' in next_line:
                    try:
                        land_cover, n_val = next_line.rsplit(",", 1)
                        table_data.append([current_table_number, land_cover.strip(), float(n_val.strip())])
                    except ValueError:
                        pass
                j += 1
            break

    return pd.DataFrame(table_data, columns=["Table", "Land Cover Type", "n"])


Example Usage from Previous Notebook

The function is included above, and all code that was specific to the previous notebook is placed here for reference: 
```
## === User-defined file paths ===
input_file = Path(r"C:\GH\ASFPM-LLM-Data-Management-Workshop\Data\Ascii\RAS_BaldEagleDamBrk.g09")
output_lcmann_csv = "LCMann_Table.csv"
output_lcmann_region_csv = "LCMann_Region_Table.csv"

## === Read geometry file ===
with open(input_file, "r", encoding="utf-8", errors="ignore") as file:
    lines = file.readlines()

## === Parse both tables ===
lcmann_df = parse_lcmann_table(lines)
lcmann_region_df = parse_lcmann_region_table(lines)

## Display both tables
print("LCMann Table:")
display(lcmann_df)
print("\nLCMann Region Table:")
display(lcmann_region_df)

## === Export to CSV ===
lcmann_df.to_csv(output_lcmann_csv, index=False)
lcmann_region_df.to_csv(output_lcmann_region_csv, index=False)

print(f"Saved LCMann Table to: {output_lcmann_csv}")
print(f"Saved LCMann Region Table to: {output_lcmann_region_csv}")
```

In [4]:
def write_lcmann_table(df, lines):
    new_lines = lines.copy()
    for i, line in enumerate(lines):
        if line.startswith("LCMann Table="):
            j = i + 1
            while j < len(lines):
                current = lines[j].strip()
                if current.startswith("LCMann") or not current or "=" in current:
                    break
                j += 1
            # Replace the block between i+1 and j with updated lines
            updated_block = [f"{row['Land Cover Type']},{row['n']:.3f}\n" for _, row in df.iterrows()]
            new_lines[i+1:j] = updated_block
            break
    return new_lines

# NOTE: We don't need LCMann Region Tables (they do not exist in this geometry)

Example Usage from Previous Notebook

The function is included above, and all code that was specific to the previous notebook is placed here for reference: 
```
# 🧪 Example: increase all 'n' values by 0.02
lcmann_df['n'] += 0.002
print("Revised LCMann Table:")
display(lcmann_df)

# 🛠 Update the file content
updated_lines = write_lcmann_table(lcmann_df, lines)

# 💾 Write updated geometry back to disk (overwrite or save as new)
output_file = Path(r"RAS_BaldEagleDamBrk_MODIFIED.g09")
with open(output_file, "w", encoding="utf-8") as f:
    f.writelines(updated_lines)

print(f"Updated geometry file saved to: {output_file}")
```

-----

# RAS-Commander Installation and Import

In [5]:
# If you haven't installed ras-commander, remove the leading # on the next line:
#!pip install --upgrade ras-commander

# Import all functions and classes from ras-commander
from ras_commander import *

In [6]:
# All Other Imports (consolidate all imports here)
import os
import shutil

# Project Setup 

All outputs of this notebook should be placed in the subfolder "RAS-Commander Mannings Sensitivity Test".  

If the folder exists, delete it.  Otherwise, create it.  This will make our notebook repeatable. 

This folder should be sensitivity_folder and should be set as the current working direcory to simplify paths in the notebook. 


## Extract RAS Example Project with RASExamples

The RasExamples.extract_project function will download the HEC Example projects zip file and extracts the specified example project by name.  The function creates the subfolder "example_projects" and places the example project in that subfolder.  The function returns the path so we can easily initialize it.

In [7]:
# Due to slow download speeds on the shared wifi, download this .zip that contains only the BaldEagleCrkMulti2D Project:




In [ ]:
# Extract RasExamples.exract_project "BaldEagleCrkMulti2D" (it will default to writing to subfolder "example_projects")
project_folder = RasExamples.extract_project("BaldEagleCrkMulti2D")

project_folder

## Initialize RAS Project with RAS-Commander

This function reads the HEC-RAS PRJ, Plan, Unsteady and RASMapper files to retrieve data into dataframes for programmatic use.  

In [ ]:
init_ras_project(project_folder, "6.6")

-----
Show all Project Dataframes

## Display Project Dataframes
(For information only, we will be using plan 18 for this demonstration)

In [ ]:
ras.plan_df

In [ ]:
ras.geom_df

In [ ]:
ras.flow_df

In [ ]:
ras.boundaries_df

In [ ]:
ras.rasmap_df

-----

## Specify Source Plan and Geometry

In [ ]:
#We are using Plan 06 as the source plan
source_plan = "18"

source_plan


In [ ]:
# using plan_df, look up geometry_number by plan_number (using source_plan)
source_geom = ras.plan_df[ras.plan_df['plan_number'] == source_plan]['geometry_number'].iloc[0]

source_geom

## Clone Source Plan and Geometry and Set Active Geometry in Cloned Plan

In [ ]:
revised_plan = RasPlan.clone_plan(source_plan)

revised_plan

In [ ]:
revised_geom = RasPlan.clone_geom(source_geom)

revised_geom

In [ ]:
# Set revised_geom as the active geometry for thew revised_plan

RasPlan.set_geom(revised_plan, revised_geom)

-----

## Revise ShortID, Project Name and Description

OPTIONAL: For revised_plan, replace the last 3 of the shortID with "REV" and put "Revised" in the project name, and "REVISED MANNINGS N VALUES" added to the description.  

In [ ]:
# Get "Plan Title" and "Short Identifier" from plan_df using source_plan

original_plantitle = ras.plan_df[ras.plan_df['plan_number'] == source_plan]['Plan Title'].iloc[0]
original_shortid = ras.plan_df[ras.plan_df['plan_number'] == source_plan]['Short Identifier'].iloc[0]

# Retrieve Description with RasPlan.read_plan_description
original_description = RasPlan.read_plan_description(source_plan)

In [ ]:
original_plantitle

In [ ]:
original_shortid

In [ ]:
original_description

In [ ]:
# Revise the plan title, shorid and description and write it back to file.  

# Create revised values
revised_shortid = original_shortid[:-3] + "REV"
revised_plantitle = "Revised " + original_plantitle
revised_description = original_description + " REVISED MANNINGS N VALUES"

# Write revised values back to plan file
RasPlan.set_plan_title(revised_plan, revised_plantitle)
RasPlan.set_shortid(revised_plan, revised_shortid) 
RasPlan.update_plan_description(revised_plan, revised_description)


# Go back and re-run previous cells to confirm updates if desired

-----

## Revise Manning's N using the functions above 

NOTE: There is no Mannings n regional overrides in this geometry, so we are omitting the lcmann_region_table code

In [ ]:
revised_plan

In [ ]:
# We have the geometry number, so we can easily derive the geometry path
# geometry path is project_folder + ras.project_name +.g{revised_geom}

revised_geometry_path = os.path.join(ras.project_folder, ras.project_name + ".g" + revised_geom)
revised_geometry_path

In [ ]:
# RE-USE THE PREVIOUS USAGE EXAMPLE, REVISED AS NOTED: 

## === User-defined file paths ===
input_file = revised_geometry_path # changed to revised_geometry_path
output_lcmann_csv = os.path.join(ras.project_folder, "LCMann_Table.csv") # changed to place in ras.project_folder

## === Read geometry file ===
with open(input_file, "r", encoding="utf-8", errors="ignore") as file:
    lines = file.readlines()

## === Parse both tables ===
lcmann_df = parse_lcmann_table(lines)

## Display both tables
print("LCMann Table:")
display(lcmann_df)

## === Export to CSV ===
lcmann_df.to_csv(output_lcmann_csv, index=False)
print(f"Saved LCMann Table to: {output_lcmann_csv}")


In [ ]:
# 🧪 Example: increase all 'n' values by 0.006
lcmann_df['n'] += 0.006
print("Revised LCMann Table:")
display(lcmann_df)

# 🛠 Update the file content
updated_lines = write_lcmann_table(lcmann_df, lines)

# 💾 Write updated geometry back to disk (overwrite or save as new)
output_file = revised_geometry_path                                               # Changed to revised_geometry_path
with open(output_file, "w", encoding="utf-8") as f:
    f.writelines(updated_lines)

print(f"Updated geometry file saved to: {output_file}")

## Run Both Source and Revised Plan in Parallel

In [ ]:
RasCmdr.compute_parallel([source_plan, revised_plan], num_cores=2, clear_geompre=True)

## Extract Results for Both Plans

In [ ]:
### INSERT NEW CODE AND MARKDOWN CELLS HERE

source_plan_resuls_xr = HdfResultsMesh.get_mesh_cells_timeseries(source_plan)
revised_plan_resuls_xr = HdfResultsMesh.get_mesh_cells_timeseries(revised_plan)

In [ ]:
# Display xarray information 
source_plan_resuls_xr 

In [ ]:
# Display xarray information 
revised_plan_resuls_xr 

## Compare Results and Plot Results at Mesh Cell #116 (2079571, 367903) to compare results

To make things simpler, I opened RASMapper to get the Cell #116.  

![RASMapper - Geometry .g11](images\rascmdr_rasmap_results.png)  



In [ ]:
import matplotlib.pyplot as plt

# --- Plot Comparison at Cell ---
print("Generating comparison plot for cell 116...")

# Define the cell ID and variables to compare
target_cell_id = 116

# Initialize figure
fig, ax = plt.subplots(figsize=(12, 6))

# Plot data for source plan
source_cell_ts = source_plan_resuls_xr['BaldEagleCr']['Water Surface'].sel(cell_id=target_cell_id)
ax.plot(source_cell_ts.time.values, source_cell_ts.values, 
        label=f"Original (Plan {source_plan})", 
        color='blue', linewidth=1.5)

# Add peak value line for source plan
source_peak = source_cell_ts.max().item()
ax.axhline(source_peak, color='blue', linestyle='--', alpha=0.5,
           label=f'Peak Original: {source_peak:.2f} ft')

# Plot data for revised plan  
revised_cell_ts = revised_plan_resuls_xr['BaldEagleCr']['Water Surface'].sel(cell_id=target_cell_id)
ax.plot(revised_cell_ts.time.values, revised_cell_ts.values,
        label=f"Mannings +0.01 (Plan {revised_plan})",
        color='red', linewidth=1.5)

# Add peak value line for revised plan
revised_peak = revised_cell_ts.max().item()
ax.axhline(revised_peak, color='red', linestyle='--', alpha=0.5,
           label=f'Peak Mannings +0.06: {revised_peak:.2f} ft')

# Customize the plot
ax.set_title(f"Water Surface Elevation Comparison at Cell {target_cell_id}")
ax.set_xlabel("Time") 
ax.set_ylabel("Water Surface Elevation (ft)")
ax.grid(True, linestyle='--', alpha=0.6)
ax.legend()

plt.tight_layout()
plt.show()
